In [ ]:
import itertools
import pandas as pd
import numpy as np
import plotly.express as px
import skimage
import scipy
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN


def bump(x):
    return np.where(np.abs(x) < 1, np.exp(1 / (x ** 2 - 1) + 1) ** 0.2, 0)

In [ ]:
fname = r"J:\ctgroup\Edward\DATA\VMI\20220613\xe005_e_calibrated.h5"
base_data = pd.read_hdf(fname, key="data")

In [ ]:
data = base_data.query("sqrt(px**2+py**2+pz**2)<0.4").sample(frac=1).reset_index(drop=True)
data = data.query("pz>0")
#
data_sym = data.copy()
data_sym['px'] = -data_sym['px']
data_sym['py'] = -data_sym['py']
data_sym['pz'] = -data_sym['pz']
data = pd.concat([data, data_sym])
del data_sym

In [ ]:
data['pr'] = np.sqrt(data['px'] ** 2 + data['py'] ** 2 + data['pz'] ** 2)
data['phi'] = np.arctan2(data['px'], data['pz'])
data['theta'] = np.arccos(data['px'] / data['pr'])
data['E'] = data.pr ** 2 / 2

h2d, re, pe = np.histogram2d(data['pr'], data['phi'], bins=1024)
re = 1 / 2 * (re[:-1] + re[1:])
pe = 1 / 2 * (pe[:-1] + pe[1:])
rr, pp = np.meshgrid(re, pe)

mask = bump((rr - 0.25) / 0.1) * 1 / rr
signal = h2d / np.sum(h2d, axis=0, keepdims=True) * mask.T
fig = px.imshow(
        h2d * mask.T, origin='lower',
        x=pe, y=re,
        height=600, width=700,
        aspect='auto',
        color_continuous_scale="Inferno"
)

signal_dog = skimage.filters.difference_of_gaussians(signal, 30, 90, mode='wrap')

px.imshow(
        signal_dog, origin='lower',
        x=pe, y=re,
        height=600, width=700,
        aspect='auto'
).show()

fig.add_trace(go.Contour(
        z=signal_dog, x=pe, y=re,
        contours_coloring='lines',
        line_color='white',
        line_width=2,
)
)

rel_mins = scipy.signal.argrelmin(signal_dog, axis=0)
p_mins, r_mins = pe[rel_mins[1]], re[rel_mins[0]]
labels = DBSCAN(.013).fit(np.vstack((p_mins, r_mins)).T).labels_
fig.add_scatter(x=p_mins, y=r_mins, mode='markers')
fig.show()
px.scatter(x=p_mins, y=r_mins, color=labels).show()

curve_df_exp = pd.DataFrame({"theta": p_mins[labels == 2], "r": r_mins[labels == 2]})
curve_df = curve_df_exp

In [ ]:
curve_df.sort_values("theta", inplace=True)
curve_df['r_smooth'] = scipy.interpolate.make_smoothing_spline(curve_df.theta, curve_df.r, lam=.0001)(curve_df.theta)
curve_df['E'] = curve_df.r_smooth ** 2 / 2
curve_df['drdt'] = np.gradient(curve_df.r_smooth, curve_df.theta)
curve_df['Group Delay'] = np.gradient(curve_df.theta, curve_df.E) * 2

e_hist, ee = np.histogram(data.pr ** 2 / 2, bins=1024, weights=bump((data.pr - 0.25) / 0.1) * 1 / data.pr)
ee = 1 / 2 * (ee[:-1] + ee[1:])
signal = scipy.interpolate.make_smoothing_spline(ee, e_hist / np.max(e_hist), lam=1e-8)

px.line(x=ee, y=e_hist).add_scatter(x=ee, y=signal(ee)).show()

curve_df['Total Signal'] = signal(curve_df.E)

fig = px.line(curve_df, x='theta', y='r_smooth')
fig.add_scatter(x=curve_df.theta, y=curve_df.r, mode='markers', marker=dict(color='red', size=2))
fig.show()
px.scatter(curve_df, x='theta', y='drdt')

px.line(curve_df, x='E', y='theta').show()
px.line(curve_df, x='E', y='Group Delay', range_x=(0.01, 0.05), range_y=(0, 300)).add_hline(y=110).show()
px.line(curve_df, x='E', y='Total Signal', range_x=(0.01, 0.05)).show()

In [ ]:
theory_file = r"D:\DATA\4ranges-volume-momspe-2d-sw.dat"
d = []
for line in open(theory_file):
    if len(line.strip().split("  ")) == 6:
        d.append([float(x) for x in line.strip().split('  ')])

d = np.asarray(d)
d = d[d[:, 0] < 0.35]


def fit_fn(theta, theta0, a, b):
    return a * np.cos((theta - theta0) * 2) + b


pr = d[:, 0]
theta = d[:, 1] % (2 * np.pi) - np.pi / 2
I = d[:, 4]

grid_size = (len(set(pr)), len(set(theta)))
pr = pr.reshape(grid_size)
r_v = np.asarray(sorted(np.unique(pr)))
theta = theta.reshape(grid_size)
theta_v = np.asarray(sorted(np.unique(theta)))
I = I.reshape(grid_size)

ia, ib = np.hsplit(I, (512,))
I += np.hstack((ib, ia))

for i in range(len(r_v)):
    I[i] = scipy.interpolate.make_smoothing_spline(theta_v, I[i], lam=1e-1)(theta_v)
mask = bump((pr - 0.2) / 0.1)
I *= mask / np.sum(I, axis=0, keepdims=True) / pr ** 2

r = np.linspace(0, len(r_v), 1024)
theta = np.linspace(0, len(theta_v), 1024)

rr, tt = np.meshgrid(r, theta)
I = scipy.ndimage.map_coordinates(I, [rr.flatten(), tt.flatten()])

r_v = np.linspace(min(r_v), max(r_v), 1024)
theta_v = np.linspace(min(theta_v), max(theta_v), 1024)
pr, theta = np.meshgrid(r_v, theta_v)
I = I.reshape(len(r_v), len(theta_v)).T

fig = px.imshow(I, origin='lower', x=theta_v, y=r_v, aspect='auto')

signal_dog = skimage.filters.difference_of_gaussians(I, 1, 100, mode='wrap')
# signal_dog=skimage.filters.meijering(signal_dog, mode='wrap')
# signal_dog=skimage.morphology.isotropic_erosion(signal_dog,3)
# signal_dog=skimage.filters.difference_of_gaussians(signal_dog, 30, 60, mode='wrap')

fig.add_trace(go.Contour(
        z=signal_dog, x=theta_v, y=r_v,
        contours_coloring='lines',
        line_color='white',
        line_width=2,
)
)

px.imshow(signal_dog, origin='lower', ).show()

# rel_mins= scipy.signal.argrelmin(signal_dog, axis=0)


# p_mins,r_mins= theta_v[rel_mins[1]], r_v[rel_mins[0]]
p_mins, r_mins = [], []
for r in range(len(r_v)):
    peaks = scipy.signal.find_peaks(-signal_dog[r])[0]
    for p in peaks:
        p_mins.append(theta_v[p])
        r_mins.append(r_v[r])

p_mins = np.asarray(p_mins)
r_mins = np.asarray(r_mins)

fig.add_scatter(x=p_mins, y=r_mins, mode='markers')
fig.show()
p_mins, r_mins = p_mins[(r_mins > 0.154) & (r_mins < 0.25)], r_mins[(r_mins > 0.154) & (r_mins < 0.25)]

labels = DBSCAN(.06).fit(np.vstack((p_mins, r_mins)).T).labels_

px.scatter(x=p_mins, y=r_mins, color=labels).show()

m = 0
curve_df_theory = pd.DataFrame({"theta": p_mins[labels == m], "r": r_mins[labels == m]})
curve_df = curve_df_theory

In [ ]:
curve_df.sort_values("theta", inplace=True)
curve_df = curve_df.groupby("theta").mean().reset_index()
curve_df['r_smooth'] = scipy.interpolate.make_smoothing_spline(curve_df.theta, curve_df.r, lam=.001)(curve_df.theta)
curve_df['E'] = curve_df.r_smooth ** 2 / 2
curve_df['drdt'] = np.gradient(curve_df.r_smooth, curve_df.theta)
curve_df['Group Delay'] = np.gradient(curve_df.theta, curve_df.E) * 2

# e_hist,ee=np.histogram(data.pr**2/2,bins=1024,weights=bump((data.pr-0.25)/0.1)*1/data.pr)
# ee=1/2*(ee[:-1]+ee[1:])
# signal=scipy.interpolate.make_smoothing_spline(ee, e_hist/np.max(e_hist), lam=1e-8)
#
# px.line(x=ee,y=e_hist).add_scatter(x=ee,y=signal(ee)).show()
#
# curve_df['Total Signal']=signal(curve_df.E)

fig = px.line(curve_df, x='theta', y='r_smooth')
fig.add_scatter(x=curve_df.theta, y=curve_df.r, mode='markers', marker=dict(color='red', size=2))
fig.show()
px.scatter(curve_df, x='theta', y='drdt')

px.line(curve_df, x='E', y='theta').show()
px.line(curve_df, x='E', y='Group Delay', range_x=(0.01, 0.05), range_y=(0, 300)).add_hline(y=110).show()
# px.line(curve_df, x='E', y='Total Signal', range_x=(0.01,0.05)).show()